In [1]:
import os
import jieba
import numpy as np
from collections import Counter
# pip install gensim
from gensim.corpora import Dictionary

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


### 讀入stopwords

In [2]:
stopwords = []
jieba.set_dictionary('./jieba_data/dict.txt.big')
jieba.load_userdict('./jieba_data/userdict.txt')

with open('./jieba_data/stopwords.txt', 'r', encoding='UTF-8') as file:
    for data in file.readlines():
        data = data.strip()
        stopwords.append(data)

Building prefix dict from C:\Users\鄒峻安\Desktop\github project\HW1--data-mining\HW2\jieba_data\dict.txt.big ...
Loading model from cache C:\Users\鄒峻安\AppData\Local\Temp\jieba.uea1b80d53a35cd1eaa4d428049c7a876.cache
Loading model cost 1.190 seconds.
Prefix dict has been built succesfully.


In [3]:
def getTokens(text):
    tokens = []
    if len(text) > 0:
        segments = jieba.cut(text,cut_all=False)
        for t in segments:
            t = t.strip()
            if not t in stopwords:
                tokens.append(t)
    return tokens

In [4]:
def getText(filename):
    with open(filename,'r') as file:
        lines = file.readlines()
        
    news_title = lines[2]
    news_body = lines[3]
    text = news_title + news_body
    return(text)    

### 取出全部文件(180個)的內容做斷詞

In [5]:
text = ''
dir_path = './ctee_TMC'
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path,file)
    if os.path.isfile(file_path) and not file.startswith('.'):
        text = text + getText(file_path)

tokens = getTokens(text)

UnicodeDecodeError: 'cp950' codec can't decode byte 0xe6 in position 53: illegal multibyte sequence

### 共有約2萬個詞

In [ ]:
print(len(tokens))

### 共有約5000個不重複詞

In [ ]:
counter = Counter(tokens)
print(len(counter))

### 取出詞頻最高前500個詞

In [ ]:
top_tokens_tuple = counter.most_common(500)
all_tokens = []
for t in top_tokens_tuple:
    all_tokens.append(t[0])

print(all_tokens)

### 過濾掉沒有在top 500裡的詞

In [ ]:
def filterTokens(tokens):
    global all_tokens
    filtered_tokens = []
    for t in tokens:
        if t in all_tokens:
            filtered_tokens.append(t)
    return filtered_tokens

In [ ]:
doc_tokens = []
dir_path = './ctee_TMC'
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path,file)
    if os.path.isfile(file_path) and not file.startswith('.'):
        wanted_tokens = []
        text = getText(file_path)
        tokens = getTokens(text)
        tokens2 = filterTokens(tokens)
        doc_tokens.append(tokens2)

### 稀疏矩陣

In [ ]:
sparse_matrix = []

dir_path = './ctee_TMC'
for file in os.listdir(dir_path):
    file_path = os.path.join(dir_path,file)
    if os.path.isfile(file_path) and not file.startswith('.'):
        text = getText(file_path)
        tokens = getTokens(text)
        doc_to_term = []
        for t in all_tokens:
            if t in tokens:
                doc_to_term.append(1)
            else:
                doc_to_term.append(0)
        sparse_matrix.append(doc_to_term)

### 共現矩陣

In [ ]:
def co_occurrence_matrix(ls):
    global matrix
    length = len(ls)
    for i in range(length):
        for j in range(length):
            if i != j:
                matrix[[ls[i]], [ls[j]]] += 1

In [ ]:
dimension = len(all_tokens)
matrix = np.matrix([[0] * dimension] * dimension)

dt = Dictionary(doc_tokens).token2id
doc_token_ids = [[dt[word] for word in words] for words in doc_tokens]

for ids in doc_token_ids:
    co_occurrence_matrix(ids)

In [ ]:
print(dt)

In [ ]:
print(matrix)